In [6]:
import os
import re
import logging
import SimpleITK as sitk
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [2]:
oar_names = ['spleen', 'kidney_right', 'kidney_left', 'gallbladder', 
             'liver', 'stomach', 'pancreas', 'prostate' ]
oar_indices = [1, 2, 3, 4, 
              5, 6, 7, 8]
external_directory = '/Volumes/My Passport/RPT/Patientwise_data_original'

Trial code - Fitting bi-phase monoexponential curve

In [76]:
spect_file_pattern = re.compile(rf"{subject_id}_D(\d+)_(\d+)_SPECT.nii")

In [77]:
subject_id = 'P00007'
subject_folder = os.path.join(external_directory, subject_id)
subject_files = []

In [78]:
for filename in os.listdir(subject_folder):
    match = spect_file_pattern.match(filename)
    if match:
        dose, timepoint = match.groups()
        subject_files.append({
            'Dose': dose,
            'Timepoint': int(timepoint),
            'Filename': filename
        })
        
subject_files = pd.DataFrame(subject_files)
subject_files = (
    subject_files.groupby('Dose')
    .filter(lambda x: len(x)==3)
    .sort_values(by=['Dose', 'Timepoint'])
    .reset_index(drop=True))

for column in oar_names:
    subject_files[column] = pd.NA

In [79]:
subject_files

,Dose,Timepoint,Filename,spleen,kidney_right,kidney_left,gallbladder,liver,stomach,pancreas,prostate
0,2,72,P00007_D2_72_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,144,P00007_D2_144_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2,168,P00007_D2_168_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,3,48,P00007_D3_48_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,3,72,P00007_D3_72_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,3,144,P00007_D3_144_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,4,24,P00007_D4_24_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,4,120,P00007_D4_120_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,4,168,P00007_D4_168_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,5,24,P00007_D5_24_SPECT.nii,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [80]:
def oar_wise_level_activity(image, segmentation):
    oar_wise_activity = {}
    for oar_index in oar_indices:
        oar_mask = segmentation==oar_index
        mean_oar_activity = image[oar_mask].mean()
        oar_wise_activity[oar_names[oar_index]].append(mean_oar_activity)
    return oar_wise_activity

In [81]:
for index, row in subject_files.iterrows(): # For each dose

    dose = row['Dose']
#     seg_filename = '{subject_id}_{dose}_seg.nii' # Re-do segmentations in this manner
    seg_filename = subject_folder+'/P00007_D1_48_seg.nii'
    segmentation = sitk.GetArrayFromImage(sitk.ReadImage(seg_filename))
    spect = sitk.GetArrayFromImage(sitk.ReadImage(subject_folder+'/'+row['Filename']))
    
    oar_activity = oar_wise_level_activity(spect, segmentation)
    for oar_name, activity in oar_activity.items():
        subject_files.at[index, oar_name] = activity

IndexError: boolean index did not match indexed array along dimension 0; dimension is 204 but corresponding boolean dimension is 151

In [82]:
def monoexponential(t, c, k):
    return c*np.exp(-k*t)

def fit_tac(activities, timepoints): ### THREE timepoints together
    fit_params_tia = pd.DataFrame(columns=['OAR', 'c1', 'k1', 'c2', 'k2', 'TIA'])
    for oar_index in range(len(oar_indices)):

        (c1, k1), _ = curve_fit(monoexponential, timepoints[:2], activities[:2, oar_index],
                               p0=[activities[0, oar_index], 0.1], bounds=(0, [np.inf,1]))
        (c2, k2), _ = curve_fit(monoexponential, timepoints[2:], activities[2:, oar_index],
                               p0=[activities[1, oar_index], 0.1], bounds=(0, [np.inf,1]))

        c2 = monoexponential(timepoints[1], c1, k1)/np.exp(-k2*timepoints[1]) #ADJUST to match at t2!

        integral_1, _ = quad(monoexponential, 0, timepoints[1], args = (c1, k1))
        integral_2, _ = quad(monoexponential, timepoints[1], np.inf, args = (c2, k2))
        tia = integral_1 + integral_2
        fit_params_tia.loc[oar_index] = [oar_index, c1, k1, c2, k2, tia]
    return fit_params_tia

In [83]:
def tia_madsen_calculation(activities, timepoint): ###ONE timepoint
    k_pop = -0.004 #this is based only on physical half life - find population-based values!
    tia_values = pd.DataFrame([a*(np.exp(k_pop*t)/(-k+pop)) for a in activities],
                             columns = oar_names)
    return tia_values

In [84]:
def tia_hanscheid_calculation(activities, timepoint): ###ONE timepoint
    tia_values = pd.DataFrame([a*(2*timepoint/np.log(2)) for a in activities], 
                              columns = oar_names)
    return tia_values